<a href="https://colab.research.google.com/github/Xinrui-Wang/IELTS-Study-Assistant/blob/main/IELTS_Task2_Writing_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

测试api是否正常

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-MgbjqB89s6uRHqhMi08SbQcGXp6Cz3REHjif6sQTgck757XQ", # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)

completion = client.chat.completions.create(
    model = "moonshot-v1-8k",
    messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你知道什么是雅思作文task1吗"}
    ],
    temperature = 0.3,
)

# 通过 API 我们获得了 Kimi 大模型给予我们的回复消息（role=assistant）
print(completion.choices[0].message.content)

当然，雅思（International English Language Testing System，国际英语语言测试系统）考试中的Task 1是学术类和培训类写作部分的第一项任务。这项任务要求考生根据给定的信息或数据，完成一篇描述性或解释性的短文。

在学术类雅思写作Task 1中，考生通常需要描述一个图表、表格或图形，这可能包括趋势、比较、分布等。考生需要展示他们能够理解和分析数据，并用清晰、准确、恰当的语言来表达这些信息。

培训类雅思写作Task 1则可能要求考生写一封书信，解决一个问题或解释一个情况。这可能包括投诉信、请求信、建议信等。

无论是学术类还是培训类，Task 1都要求考生能够：

1. **理解题目要求**：准确把握题目要求，明确写作目的。
2. **组织信息**：合理组织数据或信息，使其逻辑清晰。
3. **使用恰当的语言**：使用恰当的词汇和语法结构来描述数据或解释情况。
4. **保持客观和准确**：在描述数据时保持客观，避免主观判断。

雅思写作Task 1的评分标准包括任务完成度、连贯与衔接、词汇资源和语法范围与准确性。考生需要在这四个方面都表现出色，才能获得高分。


下面是装载图片解析所必要的库

In [1]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,988 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [2]:
pip install pytesseract

In [3]:
pip install Pillow

测试图片解析功能

In [4]:
from PIL import Image
import pytesseract

# 确保 Tesseract 路径已添加到环境变量中，如果没有，指定路径：
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Windows

# 读取图片
img = Image.open('/content/20241118155133.png')

# 进行OCR识别
text = pytesseract.image_to_string(img)

print(text)

It has been suggested that all young adults should be required to undertake a period of unpaid work
helping people in the community. Would the drawbacks of such a requirement be greater than the



# 输入预处理模块，解析输入内容。
该模块尚不完美，对图片为乱序输入的情况没有做处理。

In [5]:
from typing import List, Dict
from PIL import Image
import pytesseract
import os

def process_user_input(user_input: Dict[str, List[str]]) -> str:
    """
    处理用户输入，提取图片中的文字并与文本内容合并。

    :param user_input: 包含 images（图片路径列表）和 text（用户输入的文本）的字典。
                       示例: {"images": ["img1.jpg", "img2.jpg"], "text": "This is a text input."}
    :return: 合并后的完整文本内容。
    """
    # 校验 text 字段
    if "text" not in user_input or not user_input["text"].strip():
        raise ValueError("Text input cannot be empty.")

    # 校验 images 字段是否存在
    image_texts = []
    if "images" in user_input and user_input["images"]:
        for image_path in user_input["images"]:
            # 检查图片路径是否有效
            if not os.path.isfile(image_path):
                raise FileNotFoundError(f"Image file not found: {image_path}")

            # 提取图片中的文字
            try:
                image_text = pytesseract.image_to_string(Image.open(image_path))
                image_texts.append(image_text.strip())
            except Exception as e:
                raise RuntimeError(f"Failed to process image {image_path}: {str(e)}")

    # 合并图片文字和用户提供的文本
    combined_text = "\n".join(image_texts) + "\n" + user_input["text"]
    return combined_text.strip()

# # 示例使用
# if __name__ == "__main__":
#     input_data = {
#         "images": ["/content/20241118155133.png", "/content/20241118155138.png"],  # 替换为实际的图片路径
#         "text": "这是题目，帮我写一篇高分范文吧"
#     }
#     try:
#         result = process_user_input(input_data)
#         print("Processed text:")
#         print(result)
#     except Exception as e:
#         print(f"Error: {e}")


Processed text:
It has been suggested that all young adults should be required to undertake a period of unpaid work
helping people in the community. Would the drawbacks of such a requirement be greater than the
benefits to the community and the individual young adults?
这是题目，帮我写一篇高分范文吧


雅思Task2模块，包含图片解析与问题回答

In [9]:
from openai import OpenAI
import pytesseract
from PIL import Image

client = OpenAI(
    api_key="sk-MgbjqB89s6uRHqhMi08SbQcGXp6Cz3REHjif6sQTgck757XQ",  # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)

# 定义函数，处理英语学习问题
def IELTS_Writing_Tutor_Task2(question):
    teacher_prompt = """
    请根据用户提供的雅思Task 2作文题目，写一篇高分作文。要求如下：

    1. 结构清晰：作文应包括引言、至少两个主体段落、以及结论段。
    2. 论点明确：每个主体段落要有一个清晰的论点，并通过具体的例子或理由进行支持。
    3. 语言流畅且高级：使用复杂且准确的句型，尽量避免简单句型，确保文章语言多样化且流畅。
    4. 不要重复观点：确保每段有独立的观点，避免重复同一论点。
    5. 语法和拼写准确：确保没有语法错误和拼写错误。
    6. 总结观点并给出个人意见：结论段要总结论点并明确给出个人意见。
    7. 论据充分：论点需要有清晰的证据或例子来支持，确保论证过程有逻辑性。

    请注意，文章需要控制在300词左右，并尽量遵循雅思写作评分标准的高分要求。

    请生成文章内容，不要显示题目或段落标题，仅提供清晰的作文内容，并在输出的时候注意分段。
    """

    # 使用 Kimi API 生成回答
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",  # 使用 moonshot-v1-8k 模型
        messages=[
            {"role": "system", "content": teacher_prompt},  # 系统提示词，定义角色和行为
            {"role": "user", "content": question}  # 学生提问
        ],
        temperature=0.3,  # 温度设置，控制生成的答案的随机性
    )

    # 返回生成的作文内容
    return completion.choices[0].message.content

# # 调用函数，传入学生的问题
# student_question = "Some people say that the purpose of education is to prepare individuals to be useful to society. Others say its purpose is to prepare individuals to achieve personal ambitions. Discuss both views and give your opinion."
# answer = IELTS_Writing_Tutor_Task2(student_question)

# # 打印回答
# print(answer)


In [10]:
input_data = {
    "images": ["/content/20241118155133.png", "/content/20241118155138.png"],  # 替换为实际的图片路径
    "text": "这是题目，帮我写一篇高分范文吧"
}

try:
    # 处理用户输入，提取有效的题目文本
    question = process_user_input(input_data)
    # 如果需要，可以打印处理后的文本
    # print("Processed text:")
    # print(question)
except Exception as e:
    print(f"Error: {e}")

# 使用处理后的题目生成作文
answer = IELTS_Writing_Tutor_Task2(question)

# 打印生成的作文内容
print(answer)


The proposition that all young adults should be required to undertake a period of unpaid community service has sparked a debate on its potential drawbacks versus benefits. This essay will explore the arguments from both sides of the spectrum, ultimately weighing the impact on the community and the individual young adults.

On one hand, the benefits of mandatory community service for young adults are manifold. Firstly, it instills a sense of social responsibility and empathy in the youth, which is crucial for the development of well-rounded individuals. Engaging in unpaid work can expose them to the diverse needs of their community, fostering a deeper understanding and compassion towards others. For instance, young adults working in homeless shelters or food banks gain firsthand experience of social issues, which can motivate them to become advocates for change in the future. Additionally, such experiences can enhance their employability by developing transferable skills such as teamwor